Нужно каким-то образом загрузить данные. И попробовать нарисовать на них bouding box-ы

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import torchvision.utils as utils 


%matplotlib inline
import matplotlib.pyplot as plt

In [47]:
import os
import json
import natsort

bboxes = []
# load json
with open('team_classification_data\\bboxes.json') as bboxes_json:
    bboxes = json.load(bboxes_json)

# load images 
data_dir = 'team_classification_data\\frames\\'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.jpeg']
sorted_filenames = natsort.natsorted(filenames)

image_tensors = [torchvision.io.read_image(data_dir + name) for name in sorted_filenames]
batch = torch.stack(image_tensors)
print(batch.shape, len(bboxes), type(bboxes))

# iterate over images an draw corresponding bboxes
# for key, value in bboxes.items():
#    print(key, value)

for bb, fn, tens in zip(bboxes.items(), sorted_filenames, image_tensors):
    print(f'{bb[0]}  - {fn}')

    bboxes_list = []
    for player_id, bbox in bb[1].items():
        # print(f'id: {player_id} - team: {bbox["team"]} - bbox: {bbox["box"]}')
        # print(tens.shape) = torch.Size([3 = rgb, 720 = height, 1280 = width]) 
        height = tens.shape[1]
        width = tens.shape[2]

        x = bbox['box'][0] * width
        y = bbox['box'][1] * height
        w = bbox['box'][2] * width
        h = bbox['box'][3] * height

        bboxes_list.append([x, y, w, h])

        # bbox_tensor = torch.tensor([x, y, w, h])
        # bbox_tensor = bbox_tensor.unsqueeze(0)


        # bbox_tensor. = torchvision.ops.box_convert(bbox_tensor, 'xywh', 'xyxy')


        # color = 'green'
        # if bbox['team'] == 0 :
        #     color = 'red'
        # img = utils.draw_bounding_boxes(tens, bbox_tensor, width=5, colors=color)

        # if player_id > '4':
        #     img = transforms.ToPILImage()(img)
        #     img.show()
    
    bboxes_tensor = torch.tensor(bboxes_list)
    print(bboxes_list, bboxes_tensor)
    bboxes_tensor = bboxes_tensor.unsqueeze(0)
    bboxes_tensor = torchvision.ops.box_convert(bboxes_tensor, 'xywh', 'xyxy')
    print(bboxes_tensor)
    print(tens)

    img = utils.draw_bounding_boxes(tens, bboxes_tensor, width=2, colors='green')
    img = transforms.ToPILImage()(img)
    img.show()

    break





torch.Size([100, 3, 720, 1280]) 100 <class 'dict'>
1310  - 1310.jpeg
[[147.6285577889684, 255.99795244419116, 81.82212034100837, 152.63871900698197], [871.8487127118674, 313.4508504284022, 130.96409851161883, 221.54453004544163], [1126.18748843893, 244.62134911372516, 84.13255687486667, 198.16794911740126], [621.0183320115026, 276.46175937213104, 91.12088758939889, 208.38885758656093], [668.0268583822732, 356.5367737878781, 133.98136663257037, 241.27491512951957], [686.1048754181467, 247.97988138642003, 86.32852027329228, 182.28439135598308], [808.638884818095, 218.1205706183819, 73.33930536223944, 174.99381857469814], [511.2910386499924, 182.4466164592711, 72.20012244379666, 144.90390676246537], [336.4641105980184, 231.0954725183006, 78.18405010030374, 144.8020613107186], [544.848820300097, 206.27356823646772, 69.27584341248624, 122.79919267768803]] tensor([[ 147.6286,  255.9980,   81.8221,  152.6387],
        [ 871.8487,  313.4508,  130.9641,  221.5445],
        [1126.1875,  244.6214

ValueError: Boxes need to be in (xmin, ymin, xmax, ymax) format. Use torchvision.ops.box_convert to convert them